In [ ]:
import requests
import pandas as pd
from datetime import datetime

# Constants
BASE_URL = 'https://api.binance.com'
CRYPTOS = {
    'BTC': 'Bitcoin', 'ETH': 'Ethereum', 'HBAR': 'Hedera', 'BNB': 'BNB',
    'SOL': 'Solana', 'XRP': 'Ripple', 'RUNE': 'THORChain', 'DOGE': 'Dogecoin',
    'ADA': 'Cardano', 'AVAX': 'Avalanche', 'SHIB': 'SHIBA INU',
    'DOT': 'Polkadot', 'LINK': 'ChainLink', 'GRT': 'The Graph',
    'TRX': 'TRON', 'MATIC': 'Polygon', 'BCH': 'Bitcoin Cash',
    'ICP': 'Internet Computer', 'NEAR': 'NEAR Protocol', 'UNI': 'Uniswap',
    'VET': 'VeChain', 'LTC': 'Litecoin', 'XLM': 'Stellar', 'STX': 'Stacks',
    'FIL': 'Filecoin', 'ATOM': 'Cosmos', 'ETC': 'Ethereum Classic',
    'INJ': 'Injective', 'IMX': 'Immutable X', 'RNDR': 'Render Token'
}

def get_historical_data(symbol, start_date, end_date):
    url = f"{BASE_URL}/api/v3/klines"
    start_timestamp = int(datetime.strptime(start_date, '%Y-%m-%d').timestamp() * 1000)
    end_timestamp = int(datetime.strptime(end_date, '%Y-%m-%d').timestamp() * 1000)
    params = {
        'symbol': symbol + 'USDT',  # Assuming we're looking for the price in USDT
        'interval': '1d',
        'startTime': start_timestamp,
        'endTime': end_timestamp
    }
    response = requests.get(url, params=params)
    data = response.json()
    # Ensure we got a valid response
    if isinstance(data, dict) and data.get('msg'):
        raise Exception(f"Error fetching data for {symbol}: {data['msg']}")
    return [(datetime.fromtimestamp(item[0] / 1000).strftime('%Y-%m-%d'), float(item[4])) for item in data]

def main():
    all_data = []
    for symbol, name in CRYPTOS.items():
        print(f"Fetching data for {symbol} ({name})")
        symbol_data = get_historical_data(symbol, '2022-02-01', '2022-08-01')
        for date, price in symbol_data:
            all_data.append({'Date': date, 'Symbol': symbol, 'Price': price})

    # Convert to DataFrame
    df = pd.DataFrame(all_data)
    pivot_df = df.pivot(index='Date', columns='Symbol', values='Price').reset_index()

    # Save to CSV
    pivot_df.to_csv('crypto_daily_prices.csv', index=False)
    print("Data has been saved to crypto_daily_prices.csv")

if __name__ == "__main__":
    main()


In [ ]:
# Load the Excel file
file_path = 'C:/Users/Bowen/Desktop/crypto_and_oil_log_return_by_month.xlsx'
xls = pd.ExcelFile(file_path)

# Create Excel writer objects for standardized and correlation matrices
output_std_path = 'C:/Users/Bowen/Desktop/standardized_matrices11.xlsx'
output_corr_path = 'C:/Users/Bowen/Desktop/correlation_matrices11.xlsx'
writer_std = pd.ExcelWriter(output_std_path)
writer_corr = pd.ExcelWriter(output_corr_path)

# Corrected script to use .close() method for saving Excel files
def standardize_matrix(df):
    # Exclude non-numeric columns
    numeric_df = df.select_dtypes(include=[np.number])
    return (numeric_df - numeric_df.mean()) / numeric_df.std()

def compute_correlation_matrix(standardized_df):
    return standardized_df.corr()

# Process each sheet and use .close() method to save Excel files
try:
    for sheet_name in xls.sheet_names:
        # Shorten sheet names
        shortened_sheet_name = sheet_name.replace('Days_', '')[:20]
        std_sheet_name = f'{shortened_sheet_name}stad'
        corr_sheet_name = f'{shortened_sheet_name}corr'

        # Read each sheet
        df = pd.read_excel(xls, sheet_name)

        # Standardize the matrix
        standardized_df = standardize_matrix(df)

        # Compute correlation matrix
        correlation_matrix = compute_correlation_matrix(standardized_df)

        # Save to the respective Excel files
        standardized_df.to_excel(writer_std, sheet_name=std_sheet_name)
        pd.DataFrame(correlation_matrix).to_excel(writer_corr, sheet_name=corr_sheet_name)

    # Close the output files
    writer_std.close()
    writer_corr.close()
    result = (output_std_path, output_corr_path)
except Exception as e:
    result = str(e)

result


In [1]:
import pandas as pd
import numpy as np
from scipy.linalg import eigh
from scipy.optimize import minimize
import calendar
from scipy.linalg import LinAlgError

def marchenko_pastur_pdf(lam, Q, sigma=1):
    a = sigma**2 * (1 - np.sqrt(Q))**2
    b = sigma**2 * (1 + np.sqrt(Q))**2
    return (Q / (2 * np.pi * sigma**2 * lam)) * np.sqrt((b - lam) * (lam - a))

def negative_log_likelihood(Q, eigenvalues, sigma=1):
    a = sigma**2 * (1 - np.sqrt(Q))**2
    b = sigma**2 * (1 + np.sqrt(Q))**2
    filtered_eigenvalues = eigenvalues[(eigenvalues >= a) & (eigenvalues <= b)]
    pdf_values = marchenko_pastur_pdf(filtered_eigenvalues, Q, sigma)
    epsilon = 1e-10
    return -np.sum(np.log(pdf_values + epsilon))

# Load the Excel file
file_path = 'C:/Users/Bowen/Desktop/correlation_matrices1.xlsx'
xls = pd.ExcelFile(file_path)

# Estimate Q_MLE for each correlation matrix
# Enhanced error handling and alternative minimization strategies
Q_MLE_estimates = {}
errors = {}

for sheet_name in xls.sheet_names:
    year, month = map(int, sheet_name.split('corr')[0].split('-'))
    T = calendar.monthrange(year, month)[1]
    N = 30
    Q_init = T / N

    df = pd.read_excel(xls, sheet_name)

    try:
        eigenvals, _ = eigh(df)
        result = minimize(negative_log_likelihood, Q_init, args=(eigenvals,), bounds=[(0.001, None)])

        if result.success:
            Q_MLE_estimates[sheet_name] = result.x[0]
        else:
            # Try a different minimization method if the standard method fails
            result_alt = minimize(negative_log_likelihood, Q_init, args=(eigenvals,), bounds=[(0.001, None)], method='Nelder-Mead')
            if result_alt.success:
                Q_MLE_estimates[sheet_name] = result_alt.x[0]
            else:
                # Record error details if both methods fail
                errors[sheet_name] = {
                    'std_method_message': result.message,
                    'alt_method_message': result_alt.message
                }

    except LinAlgError as e:
        errors[sheet_name] = {'error_message': str(e)}

(Q_MLE_estimates, errors)

({'2022-02corr': 1.054283854166667,
  '2022-03corr': 1.0353064804054035,
  '2022-04corr': 1.0,
  '2022-05corr': 1.061509248830538,
  '2022-06corr': 1.0,
  '2022-07corr': 1.047038539918669,
  '2022-08corr': 1.039453293583349,
  '2022-09corr': 1.0,
  '2022-10corr': 1.0089127604166668,
  '2022-11corr': 1.0,
  '2022-12corr': 1.0536620209785978,
  '2023-01corr': 1.0322716524635827,
  '2023-02corr': 1.067135416666667,
  '2023-03corr': 1.0318509834402223,
  '2023-04corr': 1.0117987153638763,
  '2023-05corr': 1.0200634765625,
  '2023-06corr': 1.0525417508383936,
  '2023-07corr': 1.0361928655411503,
  '2023-08corr': 1.0557368154059752,
  '2023-09corr': 1.0,
  '2023-10corr': 1.0181966145833334,
  '2023-11corr': 1.0,
  '2023-12corr': 1.975978428389964,
  '2024-01corr': 1.0389148085944575},
 {})

In [ ]:
import pandas as pd
import numpy as np
from scipy.linalg import eigh

# Function to calculate lambda_plus (λ_max) using Marcenko-Pastur law
def lambda_plus(Q, sigma=1):
    return sigma**2 * (1 + np.sqrt(1/Q))**2

# Function to calculate lambda_minus (λ_min) using Marcenko-Pastur law
def lambda_minus(Q, sigma=1):
    return sigma**2 * (1 - np.sqrt(1/Q))**2

# Function to denoise each correlation matrix
def denoise_matrix(matrix, Q_MLE):
    N = matrix.shape[0]
    sigma = 1  # Assuming sigma is 1 for the Marcenko-Pastur law
    lam_plus = lambda_plus(Q_MLE, sigma)
    lam_minus = lambda_minus(Q_MLE, sigma)
    threshold = (lam_plus + lam_minus) / 2

    # Calculate eigenvalues and eigenvectors
    eigenvalues, eigenvectors = eigh(matrix)
    
    # Identify noise eigenvalues and replace them with the threshold
    noise_flags = eigenvalues < lam_plus
    eigenvalues[noise_flags] = threshold
    
    # Renormalize the eigenvalues so that their sum equals N
    eigenvalues *= N / np.sum(eigenvalues)
    
    # Construct the denoised matrix
    denoised_matrix = np.dot(eigenvectors, np.dot(np.diag(eigenvalues), eigenvectors.T))

    return denoised_matrix

# Read the Excel file
xls = pd.ExcelFile('C:/Users/Bowen/Desktop/correlation_matrices1.xlsx')

# Q_MLE for each sheet as provided
Q_MLE_dict = {
  '2022-02corr': 1.054283854166667,
  '2022-03corr': 1.0353064804054035,
  '2022-04corr': 1.0,
  '2022-05corr': 1.061509248830538,
  '2022-06corr': 1.0,
  '2022-07corr': 1.047038539918669,
  '2022-08corr': 1.039453293583349,
  '2022-09corr': 1.0,
  '2022-10corr': 1.0089127604166668,
  '2022-11corr': 1.0,
  '2022-12corr': 1.0536620209785978,
  '2023-01corr': 1.0322716524635827,
  '2023-02corr': 1.067135416666667,
  '2023-03corr': 1.0318509834402223,
  '2023-04corr': 1.0117987153638763,
  '2023-05corr': 1.0200634765625,
  '2023-06corr': 1.0525417508383936,
  '2023-07corr': 1.0361928655411503,
  '2023-08corr': 1.0557368154059752,
  '2023-09corr': 1.0,
  '2023-10corr': 1.0181966145833334,
  '2023-11corr': 1.0,
  '2023-12corr': 1.975978428389964,
  '2024-01corr': 1.0389148085944575
}

# Denoise each matrix and store the results
denoised_matrices = {}

for sheet_name in xls.sheet_names:
    # Read the correlation matrix from the current sheet
    matrix = pd.read_excel(xls, sheet_name).values

    # Get the corresponding Q_MLE value
    Q_MLE = Q_MLE_dict.get(sheet_name)

    # Skip sheets that do not have a Q_MLE value
    if Q_MLE is None:
        continue

    # Denoise the matrix
    denoised_matrices[sheet_name] = denoise_matrix(matrix, Q_MLE)

# Write denoised matrices to an Excel file
with pd.ExcelWriter('C:/Users/Bowen/Desktop/denoised_correlation_matrices11.xlsx') as writer:
    for name, matrix in denoised_matrices.items():
        pd.DataFrame(matrix).to_excel(writer, sheet_name=name)

# Return the path to the created Excel file
'C:/Users/Bowen/Desktop/denoised_correlation_matrices11.xlsx'